# Dependencies

In [12]:
from dotenv import load_dotenv
load_dotenv()

import os
import sys
import time

# CrewAI
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

from pydantic import BaseModel,Field
from crewai import Crew, Process
from langchain_openai import ChatOpenAI



OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

In [13]:
import warnings
warnings.filterwarnings('ignore')

# Tools
Now, we will use the serper dev tool and scrape tool. The serper dev tool will search google for the given query and will fetch us the websites. 
After that, we will use the scrape tool to scrape the data from the websites. This way, we will be able to get the relevant context correctly 
from the websites.

In [3]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Agents
Now, our task is to automate an event planning task, which can be done with the help of agents. 

In [4]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [5]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

# Tasks

In [8]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [9]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [10]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [11]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

# Crew

In [17]:
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4o", 
                           api_key=OPENAI_API_KEY,
                           temperature=0.5),
    process=Process.hierarchical,
    verbose=True
)

2025-02-24 16:57:31,729 - 8181330496 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Execution

In [24]:
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'High',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [26]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


# Agent: Crew Manager
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"AAPL current market data\"}"
## Tool Output: 
{'searchParameters': {'q': 'AAPL current market data', 'type': 'search', 'num': 10, 'engine': 'google'}, 'knowledgeGraph': {'title': 'Apple', 'type': 'Technology company', 'website': '', 'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwGQRv5TjjkycpctY66mOg_e2-npacrmjAb6_jAWhzlzkFE3OTjxyzbA&s=0', 'description': '', 'descriptionSource': '', 'descriptionLink': '', 'attributes': {'CEO': 'Tim Cook (Aug 24, 2011–)', 'Customer service': '1 (800) 275-2273', 'Founded': 'April 1, 1976, Los Altos, CA', 'Headquarters': 'Cupertino, CA', 'Sales': '1 (800) 692-7753', 'Subsidiaries': 'Apple Store, Beats Electronics, Apple Studios, 

In [28]:
from IPython.display import Markdown
Markdown(result.raw)

Risk Analysis Report for AAPL Trading Strategies

Introduction:
The following report evaluates risks associated with various proposed trading strategies for AAPL (Apple Inc.), given its current market conditions with a stock price of approximately $245.55 and significant trading volume. The report further suggests mitigation strategies to manage and minimize these risks.

1. Momentum Trading Risks:
   - Potential Risk: Over-reliance on technical indicators like MACD and RSI could lead to false signals, especially in volatile markets. This can result in untimely entries or exits, leading to potential losses.
   - Mitigation Strategy: Utilize additional confirmation signals such as volume analysis and chart patterns to reduce false signals. Implement disciplined entry and exit strategies with pre-defined thresholds.

2. Scalping Risks:
   - Potential Risk: Scalping requires rapid decision-making and execution, which increases the risk of errors due to latency or unexpected price shifts.
   - Mitigation Strategy: Invest in advanced trading platforms with low latency to ensure fast and accurate order execution. Implement strict risk management rules with predefined profit targets and stop-loss levels to lock in gains and prevent excessive losses.

3. High-Frequency Trading (HFT) Risks:
   - Potential Risk: HFT involves technology and infrastructure risks, including system failures or delays. Additionally, regulatory changes can impact HFT practices.
   - Mitigation Strategy: Regularly update and maintain trading systems and algorithms to ensure optimal performance. Stay informed about regulatory developments and implement compliance measures accordingly. Diversify HFT strategies to reduce dependency on a single approach.

4. Volatility Breakout Strategy Risks:
   - Potential Risk: Breakout strategies can result in false breakouts, leading to losses if price reverses shortly after entry.
   - Mitigation Strategy: Use a combination of technical indicators (Bollinger Bands, ATR) and increase stop-loss levels to give trades more breathing room. Monitor news closely for events that could invalidate breakout patterns.

Execution Timing Risks:
- Potential Risk: Poor timing of trade execution can lead to missing out on favorable conditions or getting caught in adverse market movements.
- Mitigation Strategy: Develop a structured trading plan that includes specific rules for different market sessions. Regularly review execution processes to ensure they align with current market conditions and news cycle.
  
Tight Stop-Loss Implementation Risks:
- Potential Risk: Overly tight stop-loss orders can result in premature exits before capturing significant price movements.
- Mitigation Strategy: Balance stop-loss levels to allow for market fluctuations while protecting against severe downturns. Review and adjust stop-loss placements regularly based on market dynamics and trading performance analysis.

Conclusion:
In conclusion, while the proposed trading strategies offer potential opportunities, they also come with inherent risks that require careful management. Traders should combine multiple mitigation strategies, including advanced trading systems, diversified approaches, and strict risk management protocols, to navigate the complexities of trading AAPL under current market conditions. Tight stop-loss orders and continual monitoring of market indicators are essential to protect against adverse movements while maximizing potential gains.
```

# Conclusion
With this, we can see how we can create a team of agents which can work together to achieve a common goal. This acts an introduction to the concept of tools and how it can be used with agents to help agents give better outputs.